# Interprete C-Python



## Gestione della memoria
I linguaggi dinamici seguono la filosofia Java: il sistema (Run-time Environment) mette a disposizione un gestore automatico della memoria.
Alloca e dealloca a tempo di esecuzione.
La gestione automatica __è particolarmente complessa__ _nei linguaggi a tipizzazione dinamica_.

### Garbage collector
Forma più usata per la gestione automatica della memoria.

La maggioranza dei garbage collector segue la strategia detta __tracing garbage collection__:
-   Traccia gli oggetti ancora referenziabili attraverso __una catena di riferimenti__ che parte da oggetti 'radice'

__Raggiungibilità__
In maniera ricorsiva se raggiungibile per se stesso o se esiste un riferimento ad esso tramite un oggetto esso stesso raggiungibile.

La raggiungibilità può essere di due tipi:
-   Diretta: una variabile contiene l'oggetto. Gli oggetti radice sono direttamente raggiungibili
-   Indiretta: una variabile contiene un “puntatore” all'oggetto

La non raggiungibilità di un oggetto può avere due cause:
-   Sintattica (syntactic garbage). Oggetti non più raggiungibili per vincoli sintattici: (es. riassegno un puntatore o un riferimento)
-   Semantica (semantic garbage). Oggetti non più raggiunti per via del flusso del codice eseguito: es. branch di codice mai utilizzato a run time

Non esiste algoritmo in grado di identificare i __semantic garbage__ per ciascun possibile input.

Gli algoritmi di garbage collection possono essere richiamati periodicamente (es. Java) o essere attivati sulla base di una soglia (es. Python).

Nei sistemi a soglia un ciclo è attivato quando il collector è notificato che il sistema ha bisogno di memoria.
Esempi di algoritmi più usati:
-   Algoritmo naive __Mark and Sweep__
    -   Inefficiente, set dati scandito più volte
-   Algoritmo __Tri-color Marking__
    -   Evoluzione del precedente

#### Mark and Sweep
Ogni oggetto in memoria è associato a un flag (un bit) riservato per le attività di garbage collection.
Il flag assume i valori:
-   0: oggetto non raggiungibile (default)
-   1: oggetto riconosciuto come raggiungibile

Ogni oggetto viene scandito una volta: se è raggiungibile come radice o da un oggetto radice si imposta il suo flag a 1.
Il set viene scandito una seconda volta: si libera la memoria per gli oggetti con flag a 0.

__Limiti__
-   poco performante
-   L'intero insieme degli oggetti deve essere scandito più volte linearmente

#### Tri-color Marking
In una prima fase, gli oggetti si suddividono in tre insiemi:
-   White set : tutti gli oggetti che sono referenziati a livello radice
-   Grey set : gli altri oggetti
-   Black set : vuoto
N.B gli oggetti possono passare solo in questa direzione White --> Grey --> Black

Nella seconda fase, si sceglie un oggetto O dal Grey set.
O viene messo nel Black set, perchè non deve essere rimosso.
Si identificano tutti gli oggetti che sono referenziati direttamente da 0, e quelli appartenenti al White set andranno nel Grey set.
Questo viene ripetuto fino a quando il Grey set si svuota.

Terza fase dell'algoritmo, gli oggetti nel White set sono dimostrabilmente:
-   Non raggiungibili direttamente (prima fase)
-   Non raggiungibili indirettamente (seconda fase)
=> __candidati alla rimozione__

__Pregi__
-   non richiede l'interruzione completa dell'interprete.
-   Non ci sono molteplici scansioni dell'intero set di oggetti.

### Generational Garbage Collector
Due modalità possibili per il rilascio di memoria:
-   rilasciare gli oggetti irraggiungibili _senza spostarli_ (GC in non movimento)
-   __copiare__ tutti gli oggetti raggiungibili in una __nuova area__ di memoria (GC in movimento)

Vantaggi:
-   Grandi regioni contigue di memoria disponibili
-   Ottimizzazioni possibili (mettendo vicini gli oggetti utilizzati sarà più facile trovarli dopo)

__Ipotesi generazionale__: _gli oggetti creati __più di recente__ hanno la maggiore probabilità di __diventare irraggiungibili__ nell'immediato futuro_ (mortalità infantile)

Un __GC generazionale__ divide gli oggetti per generazioni (diversa “vecchiaia”), e nei vari cicli di esecuzione, fa controlli frequenti __solo sugli oggetti delle generazioni più giovani__.
Contemporaneamente tiene traccia della creazione di riferimenti.

Se l'ipotesi generazionale è corretta, l'algoritmo può essere molto veloce anche se leggermente impreciso (potrebbe sfuggire oggetti più vecchi non più raggiungibili).

La memoria heap viene divisa in:
-   Eden: oggetti appena creati
-   Survivor Space 1 e 2: oggetti sopravvissuti ai cicli garbage collection precedenti
-   Old generation: oggetti più vecchi
-   Perm: oggetti permanenti usati dalla VM (definizione di classi e metodi)
![Alt text](media/generational_gc.png)

Ogni volta che __una regione tende a riempirsi__, viene invocato un ciclo di Garbage Collector:
-   Gli oggetti __raggiungibili__ della regione vengono copiati nella __regione successiva__ (più “anziana”)
-   Gli oggetti __irraggiungibili__ vengono __eliminati__
-   La regione viene svuotata e può essere utilizzata per l'allocazione di nuovi oggetti.
Tranne per #SS1 dove gli oggetti rimarrano lì per qualche ciclo.

__Osservazioni__
-   le generazioni giovani tendono a riempirsi velocemente.
-   realizza una garbage collection incrementale e veloce.
-   l'approccio euristico non è ottimale. Quindi è preferibile un approccio ibrido con cicli occasionali di “major garbage collection”

#### Major garbage collection
I termini “minor cycle” e “major cycle” sono spesso usati per riferirsi ai diversi cicli di garbage collection negli approcci ibridi.
Per i cicli di major collection solitamente viene usato un algoritmo Mark-and-Sweep.
Nei minor cycle usato approccio in movimento.

### Reference Counting
A ciascun oggetto viene associato __un contatore degli oggetti che lo stanno referenziando__.
Quando il contatore scende a 0 significa che __nessuno sta più referenziando l'oggetto__, che può __essere eliminato__.

__Pro__:
-   ancora più veloce
-   migliore reattività: quando si esce da un blocco di codice, il reference count della variabile descrittore va a 0.

__Contro__:
-   mantenere e gestire un contatore per ogni oggetto.
-   problema dei riferimenti circolari.
![Alt text](media/riferimenti_circolari.png)

__Problema riferimenti circolari (A<->B)__: se A esce dal suo scope non può essere eliminato perchè è ancora referenziato da B. (dangling reference)
_Soluzioni_: i riferimenti circolari vengono contrassegnati come __deboli__, quindi non considerati dal Reference Counting.

## Interprete C-Python
CPython è l'implementazione di riferimento del linguaggio di programmazione Python. _(scritto in C)_

    platform.python_implementation() # per capire quale interprete è in uso

Ne esistono altri:
-   PyPy (funziona bene solo con Restricted Python, [docs sulle differenze con CPython](https://doc.pypy.org/en/latest/cpython_differences.html))
-   Jython (Java)
-   IronPython (.NET)
-   ...

Il principale meccanismo di Garbage Collection in CPython è attraverso i __conteggi dei riferimenti__.

    import sys
    objx = "i.e. a string"
    sys.getrefcount( objx )
    # 2
    # Vi sono due riferimenti a objx
    # 1 -> creazione della variabile
    # 2 -> quando la passiamo alla funzione sys.getrefcount()

#### misurazione e la profilazione del consumo di memoria

    apt install python3-memory-profiler
Per testare una funzione basta usare il decoratorie @profile.
Esempio:

    from memory_profiler import profile
    import random as r

    @profile
    def foo():
        x = [x for x in range(0, 1000)]
        y = [ r.randint(1,100) for y in range(0, 2000)]
        del x
        return y
    if __name__=="__main__":
        foo()

Testare la memoria malloc: [tracemalloc](https://docs.python.org/3/library/tracemalloc.html)


### Sistemi di allocazione della memoria
[docs](https://docs.python.org/3/c-api/memory.html)

![Alt text](media/allocazione_memoria.png)

Come gestisce la memoria?
- Prima nelle versioni <=2.3, CPython usava la malloc di glibc e allocava e rilasciava direttamente la memoria libera. Creando e distriggendo gli oggetti ogni volta. (semplice, ma inefficiente)
- Dalla versione 2.4, CPython ha un allocatore di oggetti specifico.

![Alt text](media/memory_cpython.png)

In python ogni cosa è un oggetto e alcuni di esse possono contenere altri oggetti.
Questo approccio richiede molte piccole allocazioni di memoria.
Quindi per velocizzare le operazioni e ridurre la frammentazione, viene utilizzato un gestore speciale: PyMalloc.

__Schema di allocazione__
Le variabili sono solo riferimenti all'oggetto reale in memoria. Sono solo etichette, non memorizzano un valore.

![Alt text](media/var_a_b.png)

Se modifico un valore allora creo un nuovo oggetto, perchè gli interi sono immutabili (come float e stringhe).

![Alt text](media/var_a_b_2.png)

Se CPtython creasse ed eliminasse un gran numero di oggetti in questo modo, chiamamdo i metodi _malloc_ e _free_, le prestazioni sarebbero veramente scarse.
Infatti utilizza delle tecniche per ottimizzare il processo.

Inanzitutto utilizza uno schema sulla __heap privata__ del processo del programma python che include diversi allocatori:
-   0 per scopi generici (metodo malloc di CPython) --> si occupa dell'allocazione della memoria richiesta dal processo
-   +1 per memoria raw (per oggetti più grandi di 512 byte) --> Quando un processo necessita di memoria, questo interagisce con l'allocatore generico per fornire la memoria richiesta assicurandovi che ve ne sia a sufficienza per memorizzare tutti i dati.
-   +2 per oggetti inferiori o uguali a 512 byte (oggetti piccoli)
-   +3 specifici dell'oggetto (allocatori specifici per tipi di dati specifici)

![Alt text](media/heap_cpython.png)

quindi per ridurre l'overhead per piccoli oggetti (meno di  512 byte) Python sub-alloca grandi blocchi di memoria.
Gli oggetti più grandi vengono indirizzati all'allocatore C standard.

![Alt text](media/schema_alloca_memoria.png)

Quando un piccolo oggetto richiede memoria, l'allocatore richiede un grande blocco di memoria: __Arena__ (256 KB).
L'Arena viene divisa in Pool (4 KB), che a sua volta è formata da Block (da 8 a 512 byte).

![Alt text](media/arena_cpython.png)